# Calabi-Yau Geometry to Particle Spectra: Deep Learning Experiment

This notebook demonstrates the complete pipeline for learning the mapping between Calabi-Yau manifold geometry and particle physics observables using neural networks.

## Overview
- **Goal**: Train a neural network to predict particle spectra from geometric features
- **Input**: Hodge numbers, Euler characteristic, intersection numbers
- **Output**: Particle spectrum (continuous or discrete)
- **Model**: Multi-layer perceptron with dropout and batch normalization

## 1. Setup and Imports

In [ ]:
# Standard imports
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Custom modules
from data_generator import CalabiYauDataGenerator, generate_and_save_datasets
from models import create_model, CalabiYauMLP, DeepCalabiYauNet
from train import Trainer
from evaluate import ModelEvaluator

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
print(f'PyTorch version: {torch.__version__}')

## 2. Quick Start - Run Complete Pipeline

In [ ]:
# Quick start: Run the complete pipeline with one command
# This will generate data, train model, and evaluate

def run_complete_pipeline(task_type='regression', n_epochs=100):
    """Run the complete training pipeline."""
    
    # 1. Generate data if needed
    data_dir = '../data'
    Path(data_dir).mkdir(exist_ok=True)
    
    if not Path(f'{data_dir}/calabi_yau_train_{task_type}.csv').exists():
        print("Generating synthetic datasets...")
        generate_and_save_datasets(output_dir=data_dir, n_samples=5000)
    
    # 2. Load data
    train_df = pd.read_csv(f'{data_dir}/calabi_yau_train_{task_type}.csv')
    val_df = pd.read_csv(f'{data_dir}/calabi_yau_val_{task_type}.csv')
    test_df = pd.read_csv(f'{data_dir}/calabi_yau_test_{task_type}.csv')
    
    print(f"\nDataset sizes:")
    print(f"  Train: {len(train_df)} samples")
    print(f"  Val: {len(val_df)} samples")
    print(f"  Test: {len(test_df)} samples")
    
    # 3. Create model
    feature_cols = [col for col in train_df.columns if col != 'particle_spectrum']
    input_dim = len(feature_cols)
    output_dim = 1 if task_type == 'regression' else train_df['particle_spectrum'].nunique()
    
    model = create_model(
        model_type='mlp',
        input_dim=input_dim,
        output_dim=output_dim,
        task_type=task_type,
        hidden_dims=[128, 64, 32],
        dropout_rate=0.2
    )
    
    print(f"\nModel created with {sum(p.numel() for p in model.parameters()):,} parameters")
    
    # 4. Train model
    trainer = Trainer(model=model, device=device, task_type=task_type)
    
    train_loader = trainer.prepare_data(train_df, batch_size=32)
    val_loader = trainer.prepare_data(val_df, batch_size=32)
    test_loader = trainer.prepare_data(test_df, batch_size=32)
    
    print("\nStarting training...")
    history = trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        n_epochs=n_epochs,
        checkpoint_dir='../models'
    )
    
    # 5. Evaluate
    test_loss, test_metric = trainer.validate(test_loader)
    
    if task_type == 'regression':
        print(f"\nTest RMSE: {np.sqrt(test_loss):.4f}")
    else:
        print(f"\nTest Accuracy: {test_metric:.4f}")
    
    # 6. Plot results
    trainer.plot_history(save_path='../results/training_history.png')
    
    return model, history, test_loss

# Run the pipeline
model, history, test_loss = run_complete_pipeline(task_type='regression', n_epochs=50)

## 3. Detailed Analysis and Visualization

In [ ]:
# Load test data for detailed analysis
test_df = pd.read_csv('../data/calabi_yau_test_regression.csv')
feature_cols = [col for col in test_df.columns if col != 'particle_spectrum']
X_test = test_df[feature_cols].values.astype(np.float32)
y_test = test_df['particle_spectrum'].values.astype(np.float32)

# Create evaluator
evaluator = ModelEvaluator(model, device, task_type='regression')

# Get predictions
y_pred = evaluator.predict(X_test).flatten()

# Calculate metrics
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Test Performance:")
print(f"  MSE: {mse:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²: {r2:.4f}")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Predicted vs True
axes[0].scatter(y_test, y_pred, alpha=0.5, s=10)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('True Values')
axes[0].set_ylabel('Predictions')
axes[0].set_title(f'Predicted vs True (R² = {r2:.3f})')
axes[0].grid(True, alpha=0.3)

# Residuals
residuals = y_test - y_pred
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Residual Distribution')
axes[1].grid(True, alpha=0.3)

# Feature importance
importance = evaluator.plot_feature_importance(X_test[:100], feature_cols)
axes[2].bar(range(len(importance)), list(importance.values()), alpha=0.7)
axes[2].set_xlabel('Features')
axes[2].set_ylabel('Importance')
axes[2].set_title('Feature Importance')
axes[2].set_xticks(range(len(importance)))
axes[2].set_xticklabels(list(importance.keys()), rotation=45, ha='right')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/model_analysis.png', dpi=100, bbox_inches='tight')
plt.show()

## 4. Physical Insights

In [ ]:
# Analyze learned relationships
print("Key Physical Insights:")
print("="*50)

# Correlation analysis
correlations = {
    'h11': np.corrcoef(test_df['h11'], y_pred)[0,1],
    'h21': np.corrcoef(test_df['h21'], y_pred)[0,1],
    'euler_char': np.corrcoef(test_df['euler_char'], y_pred)[0,1],
    'h11-h21': np.corrcoef(test_df['h11']-test_df['h21'], y_pred)[0,1]
}

print("\nCorrelations with predicted spectrum:")
for feature, corr in sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True):
    print(f"  {feature:15s}: {corr:+.3f}")

print("\nInterpretation:")
if abs(correlations['h11-h21']) > 0.5:
    print("• The difference h^{1,1} - h^{2,1} strongly influences the particle spectrum")
if abs(correlations['euler_char']) > 0.3:
    print("• Euler characteristic χ plays a significant role in determining observables")
if importance['avg_intersection'] > 0.15:
    print("• Intersection numbers contribute substantially to the predictions")

print("\nModel successfully learned a complex mapping from geometry to physics!")

## 5. Save Results and Summary

In [ ]:
# Save comprehensive results
results = {
    'model_type': 'CalabiYauMLP',
    'task': 'regression',
    'test_metrics': {
        'mse': float(mse),
        'rmse': float(rmse),
        'r2': float(r2)
    },
    'feature_importance': importance,
    'correlations': correlations,
    'training_epochs': len(history['train_loss']),
    'best_val_loss': float(min(history['val_loss']))
}

# Save to JSON
with open('../results/experiment_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to ../results/experiment_results.json")
print("\n" + "="*50)
print("EXPERIMENT COMPLETE")
print("="*50)
print(f"\nFinal Test RMSE: {rmse:.4f}")
print(f"Final Test R²: {r2:.4f}")
print("\nThe model successfully learned to map Calabi-Yau geometry to particle spectra!")